# ctypes:2D array



In [2]:
%cd demo

F:\SEU\SEE\PySEE\home\notebook\demo


## Gauss Elimination Methods

https://www.codewithc.com/c-program-for-gauss-elimination-method/

In engineering and science, the solution of linear simultaneous equations is very important.

* Different analysis such as electronic circuits comprising invariant elements, 

* a network under steady and sinusoidal condition, 

* output of a chemical plant and finding the cost of chemical reactions in such plants require the solution of linear simultaneous equations.

In Gauss-Elimination method, these equations are solved by **eliminating the unknowns successively**.

* 周建华，陈建龙，张小向编：《几何于代数》 科学出版社，2012
  
  * 1.4 线性方程组的求解 

For this, let us first consider the following three equations:

$$\begin{align*}
&a1x + b1y + c1z = d1 \\
&a2x + b2y + c2z = d2 \\
&a3x + b3y + c3z = d3 
\end{align*}
$$

**Assuming** $a1 ≠ 0$,

* $x$ is `eliminated` from the second equation by subtracting $(a2/a1)$ times the first equation from the second equation.

* In the same way, $x$ is `eliminated` from third equation by subtracting $(a3/a1)$ times the first equation from the third equation

Then we get the new equations as:

$$\begin{align*}
&a1x + b1y + c1z = d1 \\
&b’2y + c’2z = d’2 \\
&b’3y + b’3z = d’3 \\
\end{align*}
$$

The elimination procedure is continued until **only one unknown remains in the last equation**:

$$\begin{align*}
&a1x + b1y + c1z = d1 \\
&b’2y + c’2z = d’2 \\
&c’’3z = d’’3
\end{align*}
$$

After its value is determined, the procedure is stopped. 

Gauss Elimination uses **back substitution** to get the values of $x$, $y$ and $z$ 
 
$$\begin{align*}
&z=  d’’3 / c’’3 \\
&y=(d’2 – c’2z) / b’2\\
&x=( d1- c1z- b1y)/ a1
\end{align*}
$$


## Gauss Elimination Library

* ./include/eqlinear.h

* ./src/gauss.c

* ./src/gausspivoting.c

In [4]:
%%file ./include/eqlinear.h
/*
   Gauss Methods 
*/
#ifndef EQLINEAR_H
#define EQLINEAR_H

// Gauss Elimination Method 
void gauss(int nrows, double **a, double *b, double *x);

//Augmented matrix and Pivoting
void gauss_am_pivoting(int nrows,double **a, double *x);

#endif /* EQLINEAR_H */

Overwriting ./include/eqlinear.h


### Gauss Elimination 

The C program for Gauss elimination method reduces the system to **an upper triangular matrix** from which the unknowns are derived by the use of **backward substitution method.**


In [3]:
%%file ./src/gauss.c

/*
   Gauss Elimination Method 
	nrows  - Matrix dimensions
	a - Matrix a[nrows][nrows]
	b - Right hand side vector b[nrows]
	x  - Desired solution vector
	
*/

#include <stdlib.h>
#include <stdio.h>
#include <math.h>
#include "eqlinear.h"

void gauss(int nrows, double **a, double *b, double *x)
{
    int i, j, k;
    double c;

    for (j = 0; j < nrows; j++) /* loop for the generation of upper triangular matrix*/
    {
        for (i = 0; i < nrows; i++)
        {
            if (i > j)
            {
                c = a[i][j] / a[j][j];
                for (k = 0; k < nrows; k++)
                {
                    // make the elements below the pivot elements equal to zero or elimnate the variables
                    a[i][k] -= c * a[j][k];
                }
                b[i] -=c*b[j]; 
            }
        }
    }

    /* this loop is for backward substitution*/
    x[nrows-1]=b[nrows-1]/a[nrows-1][nrows-1];
    for (i = nrows - 2; i >= 0; i--)
    {
        x[i]=b[i];    //1 make the variable to be calculated equal to the rhs of the last equation
        for (j = i + 1; j < nrows; j++)
        {
            x[i] -= a[i][j] * x[j]; //2 then subtract all the lhs values except the coefficient of the variable whose value 
        }
        x[i] /=a[i][i];  // 3 now finally divide the rhs by the coefficient of the variable to be calculated

    }
}


Writing ./src/gauss.c


### Augmented matrix and Pivoting 


**Augmented matrix and Pivoting**  can be done in Gauss Elimination method. 

This approach, combined with the back substitution, is quite general.


In [3]:
%%file ./src/gauss_am_pivoting.c

/*
  Augmented matrix and Pivoting
    nrows  - Matrix dimensions
    a - Matrix a[nrows][nrows+1]
	a[row][nrows] - Right hand side vector b[nrows]
	x  - Desired solution vector
*/

#include <stdlib.h>
#include <stdio.h>
#include <math.h>
#include "eqlinear.h"
void gauss_am_pivoting(int nrows,double **a, double *x)
{
    int i,j,k;
    double c;
           
    for (i = 0; i < nrows; i++) 
        for (k = i + 1; k < nrows; k++)
            if ((double)fabs(a[i][i]) < (double)fabs(a[k][i]))
                for (j = 0; j <=nrows; j++)
                {
                    double temp = a[i][j];
                    a[i][j] = a[k][j];
                    a[k][j] = temp;
                }
 
    // loop to perform the gauss elimination
    for (i = 0; i <nrows - 1; i++)
        for (k = i + 1; k <nrows; k++)
        {
             c = a[k][i] / a[i][i];
            for (j = 0; j <= nrows; j++)
                a[k][j] = a[k][j] - c * a[i][j]; 
        }
    
    // back-substitution
    for (i = nrows - 1; i >= 0; i--)
    {                            
        x[i] =a[i][nrows];        
        for (j = 0; j < nrows; j++)
            if (j != i)                              
                x[i] = x[i] - a[i][j] * x[j];
        x[i] = x[i] / a[i][i]; 
    }
}

Overwriting ./src/gauss_am_pivoting.c


### Using  the methods 

In [4]:
%%file ./src/gaussApp.c
/*
gcc -o ./bin/gaussApp ./src/gaussApp.c  ./src/gauss.c ./src/gauss_am_pivoting.c -I./include 

Example :
    
    周建华，陈建龙，张小向编：《几何于代数》 科学出版社，2012
    1.4 线性方程组的求解  P29-30  例1.21
*/

#include<stdio.h>
#include<stdlib.h> 
#include "eqlinear.h" 

int main()
{   
    int i,j;
    int nrows=3;
    double x[3];
    double a[3][3]={{1.0,3.0,2.0},
                   {2.0,1.0,1.0},
                   {-1.0,2.0,3.0}};
    double b[3]={-1.0,3.0,4.0};
    
    // **ptr
    double **ptr;
    ptr=(double**)malloc(sizeof(double)*nrows);  
    for(i=0;i<nrows;i++)  
    {   ptr[i]=(double*)malloc(sizeof(double)*(nrows+1)); // +1 Augmented matrix
        for(j=0;j<nrows;j++) 
            ptr[i][j]=a[i][j];
        ptr[i][nrows]=b[i];                       
    }
                               
    gauss_am_pivoting(nrows,ptr,x);
    //gauss(nrows,ptr,b,x);
    printf("\nThe solution is: ");
    for(int i=0; i<nrows; i++)
    {
        printf("\n\tx%d=%f\t",i,x[i]); /* x1, x2, x3 are the required solutions*/
    }
    // **ptr                   
    for(i=0;i<nrows;i++)  
       free(ptr[i]);
    free(ptr); 
 
    return(0);
}


Overwriting ./src/gaussApp.c


In [5]:
!gcc -mconsole -o  ./bin/gaussApp ./src/gaussApp.c  ./src/gauss.c ./src/gauss_am_pivoting.c  -I./include 

In [6]:
!.\bin\gaussApp


The solution is: 
	x0=1.200000	
	x1=-3.400000	
	x2=4.000000	


##  The Shared Library

In [7]:
%%file ./makefile-libeqlin.mk
CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR= ./include/

# Linux
# LIB=libeqlin.so 
LIB=libeqlin.dll

SRCS=$(SRCDIR)gauss.c \
	$(SRCDIR)/gauss_am_pivoting.c

# non-path filename
filename=$(notdir $(SRCS))

# the obj target of a source code using the pattern rule
OBJS=$(patsubst %.c,$(OBJDIR)%.o,$(filename))

all:$(LIB)
    
$(LIB): $(OBJS)  
	$(CC) -shared -o $(BINDIR)$@ $(OBJS) 

# the pattern rule: one step rule for multiple source files
$(OBJS):$(SRCS)
	$(CC) $(CFLAGS) -o $(OBJDIR)$(notdir $@) -c $(patsubst  %.o,$(SRCDIR)%.c,$(notdir $@))  -I$(INCDIR) 

Overwriting ./makefile-libeqlin.mk


In [8]:
!make -f makefile-libeqlin.mk

gcc -O3 -Wall -fPIC -o ./obj/gauss.o -c ./src/gauss.c  -I./include/ 
gcc -O3 -Wall -fPIC -o ./obj/gauss_am_pivoting.o -c ./src/gauss_am_pivoting.c  -I./include/ 
gcc -shared -o ./bin/libeqlin.dll ./obj/gauss.o ./obj/gauss_am_pivoting.o 


### C 

In [9]:
!gcc -c -o ./obj/gaussApp.o ./src/gaussApp.c -I./include/
!gcc -o  ./bin/gaussApp ./obj/gaussApp.o  -L./bin/ -leqlin

In [10]:
!.\bin\gaussApp


The solution is: 
	x0=1.200000	
	x1=-3.400000	
	x2=4.000000	


### Python

In [20]:
from ctypes import *

flib=cdll.LoadLibrary("./bin/libeqlin.dll")


def gauss(a,b):
    nrow=len(a)
    indata = (POINTER(c_double) * nrow)()
    for i in range(nrow):
        indata[i] =(c_double*nrow)(*a[i])
    bv=(c_double*nrow)(*b)
    xv=(c_double*nrow)()
    
    flib.gauss.restype = c_void_p
    flib.gauss.argtypes = [c_int,POINTER(POINTER(c_double)), POINTER(c_double),POINTER(c_double)]
    flib.gauss(nrow,indata,bv,xv)
    return list(xv)

def gauss_am_pivoting(a):
    nrow=len(a)
    indata = (POINTER(c_double) * nrow)()
    for i in range(nrow):
        indata[i] =(c_double*(nrow+1))(*a[i])
    xv=(c_double*nrow)()
    
    flib.gauss_am_pivoting.restype = c_void_p
    flib.gauss_am_pivoting.argtypes = [c_int,POINTER(POINTER(c_double)),POINTER(c_double)]
    flib.gauss_am_pivoting(nrow,indata,xv)
    return list(xv)

In [21]:
a=[[1.0,3.0,2.0],
   [2.0,1.0,1.0],
   [-1.0,2.0,3.0]
  ]
b=[-1.0,3.0,4.0]
x=gauss(a,b)
print(x)

au=[[1.0,3.0,2.0,-1.0],
   [2.0,1.0,1.0,3.0],
   [-1.0,2.0,3.0,4.0]
  ]
x=gauss_am_pivoting(au)
print(x)


[1.1999999999999993, -3.4, 4.0]
[1.2000000000000002, -3.4, 4.0]
